In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructField, StructType, StringType, IntegerType

In [0]:
# from pyspark.sql import SparkSession  -> not necessary in databricks

In [0]:
# spark = SparkSession.builder.getOrCreate()

In [0]:
# 1. method

data_rows = [
    (100, 'Sandeep', 20000),
    (200, 'Jashu', 14000),
    (300, 'Ramya', 30000),
    (400, 'Charan', 20000),
    (500, 'Dileep', 25000),
    (600, 'Reddy', 15000)
]

columns = ["emp_id", "emp_name", "salary"]

emp_df = spark.createDataFrame(data=data_rows, schema=columns)

emp_df.show()

In [0]:
emp_df.rdd.getNumPartitions()

In [0]:
display(emp_df)

Databricks data profile. Run in Databricks to view.

In [0]:
# 2. method

data_rows = [
    (100, 'Sandeep', 20000),
    (200, 'Jashu', 14000),
    (300, 'Ramya', 30000),
    (400, 'Charan', 20000),
    (500, 'Dileep', 25000),
    (600, 'Reddy', 15000)
]

schema = StructType([
    StructField("emp_id", IntegerType(), False),
    StructField("emp_name", StringType(), True),
    StructField("salary", IntegerType(), True),
])

emp_df2 = spark.createDataFrame(data=data_rows, schema=schema)
emp_df2.display()

In [0]:
# challenge - 1 --> create a new column called status and assign a default bool value (true)

In [0]:
emp_df = (
    emp_df2
    .withColumn("status", F.lit(True))
)
emp_df.display()

In [0]:
# challenge 2 --> rename salary column to emp_salary

emp_df = (
    emp_df
    .withColumnRenamed("salary", "emp_salary")
)
emp_df.display()

In [0]:
# example write